In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator
import spacy
import en_core_web_sm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from tqdm import tqdm, trange
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
import json
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
import os
import sys
import requests
from spacy.pipeline import SentenceSegmenter
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')
import time
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
! pip install -U nltk
corpus_bleu([[['are', 'you', 'going', 'to', 'enter', 'the', 'competition', '?']]], 
            [['i', 'i', 'the', 'the', 'the', 'the', '.', '.', '.', '.', '.', '.']], 
            weights=(0.25, 0.25, 0.25, 0.25))  

Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.5)


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

9.788429383461836e-232

In [5]:
## Set seed of randomization and working device
manual_seed = 77
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
    torch.cuda.manual_seed(manual_seed)

cuda


#Dataset download

In [0]:
# subdir = 'data'
# if not os.path.exists(subdir):
#     os.makedirs(subdir)
# subdir = subdir.replace('\\','/') # needed for Windows

# for ds in [
#     'webtext',
# #     'small-117M',  'small-117M-k40',
# #     'medium-345M', 'medium-345M-k40',
# #     'large-762M',  'large-762M-k40',
#     'xl-1542M',    'xl-1542M-k40',
# ]:
#     for split in ['train', 'valid', 'test']:
#         filename = ds + "." + split + '.jsonl'
#         r = requests.get("https://storage.googleapis.com/gpt-2/output-dataset/v1/" + filename, stream=True)

#         with open(os.path.join(subdir, filename), 'wb') as f:
#             file_size = int(r.headers["content-length"])
#             chunk_size = 1000
#             with tqdm(ncols=100, desc="Fetching " + filename, total=file_size, unit_scale=True) as pbar:
#                 # 1k for chunk_size, since Ethernet packet size is around 1500 bytes
#                 for chunk in r.iter_content(chunk_size=chunk_size):
#                     f.write(chunk)
#                     pbar.update(chunk_size)

Fetching webtext.train.jsonl: 679Mit [00:09, 70.0Mit/s]                                             
Fetching webtext.valid.jsonl: 13.6Mit [00:00, 41.7Mit/s]                                            
Fetching webtext.test.jsonl: 13.5Mit [00:00, 37.8Mit/s]                                             
Fetching xl-1542M.train.jsonl: 724Mit [00:12, 59.8Mit/s]                                            
Fetching xl-1542M.valid.jsonl: 14.4Mit [00:01, 9.29Mit/s]                                           
Fetching xl-1542M.test.jsonl: 14.6Mit [00:00, 56.9Mit/s]                                            
Fetching xl-1542M-k40.train.jsonl: 748Mit [00:09, 77.5Mit/s]                                        
Fetching xl-1542M-k40.valid.jsonl: 15.0Mit [00:00, 54.4Mit/s]                                       
Fetching xl-1542M-k40.test.jsonl: 14.6Mit [00:00, 64.5Mit/s]                                        


In [0]:
# code to explore data

data_dir = "/content/drive/My Drive/Colab Notebooks/data/"

def _load_split(data_dir, source, split, n=np.inf):
    path = os.path.join(data_dir, f'{source}.{split}.jsonl')
    texts = []
    for i, line in enumerate(open(path)):
        if i >= n:
            break
        texts.append(json.loads(line)['text'])
    return texts

text = _load_split(data_dir, "webtext", split="train")

text[:10]

["These girlfriends deserves a special mention for going that extra mile, hopefully doesn't set too many guys off on the path towards outrageous demands.\n\n1. She knows the severity of man-flu\n\n2. All fun and games is all good\n\n3. A voucher that says 'I love you'\n\n4. When arguments don't drag on forever.\n\n5. Providing everything he needs.\n\n6. Very understanding\n\n7. As awesome a gesture as this is, we are worried about this man's cooking skills.\n\n8. Nice cake\n\n8. Fair bargaining\n\n9. Excellent gift choice\n\n10. Very thoughtful",
 'LeSean McCoy going through warmups with first team offense. To my eye, does not look close to 100 percent when cutting and exploding.\n\nABOUT COOKIES\n\nTo help make this website better, to improve and personalize your experience and for advertising purposes, are you happy to accept cookies and other technologies?',
 "Tom Curran has been called up to England's Ashes squad. The 22-year-old Surrey all-rounder will fly out to Australia in the 

In [0]:
#code to label the data (webtext=1, gpt-2=0)
data_dir = "/content/drive/My Drive/Colab Notebooks/data/"

def load_split(data_dir, source, split, n=np.inf):
    webtext = _load_split(data_dir, 'webtext', split, n=n//2)
    gen = _load_split(data_dir, source, split, n=n//2)
    texts = webtext+gen
    labels = [0]*len(webtext)+[1]*len(gen)
    return texts, labels
  
texts, labels = load_split(data_dir, "xl-1542M-k40", "train")


In [0]:
# print(texts[250000])
# print(labels[250000])

#Dataframes creation

In [0]:
#create full df (500k docs)
df = pd.DataFrame(texts, columns=["text"])
df["labels"] = labels
df.head()

,text,labels
0,These girlfriends deserves a special mention f...,0
1,LeSean McCoy going through warmups with first ...,0
2,Tom Curran has been called up to England's Ash...,0
3,"We'll have turkey on the table Thursday but, a...",0
4,The 1945 Sinkings of the Cap Arcona and the Th...,0


In [0]:
#create subsets for testing code

gpt2_subset500 = df[:500]
webtext_subset500 = df[250000:250500]
subset_1000 = pd.concat([gpt2_subset500,webtext_subset500]).reset_index()
#subset_1000.to_csv(data_dir+"subset_1k.csv", index = True)

gpt2_subset5k = df[:5000]
webtext_subset5k = df[250000:255000]
subset_10k = pd.concat([gpt2_subset5k,webtext_subset5k]).reset_index()
#subset_10k.to_csv(data_dir+"subset_10k.csv", index = True)

gpt2_subset25k = df[:25000]
webtext_subset25k = df[250000:275000]
subset_50k = pd.concat([gpt2_subset25k,webtext_subset25k]).reset_index()
#subset_50k.to_csv(data_dir+"subset_50k.csv", index = True)

gpt2_subset50k = df[:50000]
webtext_subset50k = df[250000:300000]
subset_100k = pd.concat([gpt2_subset50k,webtext_subset50k]).reset_index()
#subset_100k.to_csv(data_dir+"subset_100k.csv", index = True)

gpt2_subset100k = df[:100000]
webtext_subset100k = df[250000:350000]
subset_200k = pd.concat([gpt2_subset100k,webtext_subset100k]).reset_index()
#subset_200k.to_csv(data_dir+"subset_200k.csv", index = True)


#EDA (early data analysis)

In [0]:
# code to prompt google to give me more ram
a = []
while(1):
    a.append("1")

ERROR! Session/line number was not unique in database. History logging moved to new session 63


In [6]:
path = "/content/drive/My Drive/Colab Notebooks/data/subset_100k.csv"
subset_100k = pd.read_csv(path, index_col=0).reset_index(drop=True)
subset_100k.head()

,index,text,labels
0,0,These girlfriends deserves a special mention f...,0
1,1,LeSean McCoy going through warmups with first ...,0
2,2,Tom Curran has been called up to England's Ash...,0
3,3,"We'll have turkey on the table Thursday but, a...",0
4,4,The 1945 Sinkings of the Cap Arcona and the Th...,0


In [0]:

spacy_en = spacy.load("en_core_web_sm", disable=["tagger", "parser", "ner","textcat"])

# def word_tokenize(text):
#     """
#     Tokenizes English text from a string into a list of strings (tokens)
#     """
#     return [tok.text for tok in spacy_en(text)]

# def sent_tokenize(text):
#     """
#     Tokenizes English text from a string into a list of strings (tokens)
#     """
#     return [tok.text for tok in spacy_en(text)]

spacy_en.add_pipe(spacy_en.create_pipe('sentencizer')) #adding sentence tokenizer

In [33]:
subset_100k.head()

,index,text,labels
0,0,These girlfriends deserves a special mention f...,0
1,1,LeSean McCoy going through warmups with first ...,0
2,2,Tom Curran has been called up to England's Ash...,0
3,3,"We'll have turkey on the table Thursday but, a...",0
4,4,The 1945 Sinkings of the Cap Arcona and the Th...,0


In [8]:
tokenized = spacy_en(subset_100k["text"][0])

for tok in tokenized:
  print(tok.text)
  break

for sent in tokenized.sents:
  print(sent)
  break

print(len(tokenized))

These
These girlfriends deserves a special mention for going that extra mile, hopefully doesn't set too many guys off on the path towards outrageous demands.
126


In [9]:
#takes less than 10 minues
start = time.time()
subset_100k["spacy_doc"] = subset_100k["text"].map(lambda x: spacy_en(x))
end = time.time()
print("time to tokenize (in secs):", end-start)
# df["tokenized_words"] = df["text"].map(lambda x: word_tokenize(x))
# df["tokenized_sents"] = df["text"].map(lambda x: sent_tokenize(x))
# tokens_sentences = [word_tokenize(t) for t in sent_tokenize(df["text"][0])]

time to tokenize (in secs): 396.71734619140625


In [10]:
print(len(subset_100k["spacy_doc"][0]))
print(len(list(subset_100k["spacy_doc"][0].sents)))

126
16


In [11]:
subset_100k.head()

,index,text,labels,spacy_doc
0,0,These girlfriends deserves a special mention f...,0,"(These, girlfriends, deserves, a, special, men..."
1,1,LeSean McCoy going through warmups with first ...,0,"(LeSean, McCoy, going, through, warmups, with,..."
2,2,Tom Curran has been called up to England's Ash...,0,"(Tom, Curran, has, been, called, up, to, Engla..."
3,3,"We'll have turkey on the table Thursday but, a...",0,"(We, 'll, have, turkey, on, the, table, Thursd..."
4,4,The 1945 Sinkings of the Cap Arcona and the Th...,0,"(The, 1945, Sinkings, of, the, Cap, Arcona, an..."


In [0]:
subset_100k["len_words"] = subset_100k["spacy_doc"].map(lambda x: len(x))

subset_100k["len_sents"] = subset_100k["spacy_doc"].map(lambda x: len(list(x.sents)))

In [13]:
#summary statistics for first 50k of 1542M-train-topk40 and 50k of webtext; total 100k docs

print("average words per doc:", np.mean(subset_100k["len_words"]))

print("average sents per doc:", np.mean(subset_100k["len_sents"]))

print("max words by doc:", np.max(subset_100k["len_words"]))

print("max sents by doc:", np.max(subset_100k["len_sents"]))

print("min words by doc:", np.min(subset_100k["len_words"]))

print("min sents by doc:", np.min(subset_100k["len_sents"]))

print("total vocab:", len(spacy_en.vocab))

average words per doc: 560.04114
average sents per doc: 22.24292
max words by doc: 4301
max sents by doc: 257
min words by doc: 2
min sents by doc: 1
total vocab: 14303


14303

In [30]:
words = subset_100k.spacy_doc.tolist()
print(words[0].text)

flat_list = [sublist.text for sublist in words]
len(flat_list)

These girlfriends deserves a special mention for going that extra mile, hopefully doesn't set too many guys off on the path towards outrageous demands.

1. She knows the severity of man-flu

2. All fun and games is all good

3. A voucher that says 'I love you'

4. When arguments don't drag on forever.

5. Providing everything he needs.

6. Very understanding

7. As awesome a gesture as this is, we are worried about this man's cooking skills.

8. Nice cake

8. Fair bargaining

9. Excellent gift choice

10. Very thoughtful


100000

In [18]:
words_set = set()

for sent in words:
  print(sent)
  words_set.add(sent)
  break

These girlfriends deserves a special mention for going that extra mile, hopefully doesn't set too many guys off on the path towards outrageous demands.

1. She knows the severity of man-flu

2. All fun and games is all good

3. A voucher that says 'I love you'

4. When arguments don't drag on forever.

5. Providing everything he needs.

6. Very understanding

7. As awesome a gesture as this is, we are worried about this man's cooking skills.

8. Nice cake

8. Fair bargaining

9. Excellent gift choice

10. Very thoughtful


In [0]:
subset_100k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   index      100000 non-null  int64 
 1   text       100000 non-null  object
 2   labels     100000 non-null  int64 
 3   spacy_doc  100000 non-null  object
 4   len_words  100000 non-null  int64 
 5   len_sents  100000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 4.6+ MB


In [0]:
subset_100k.describe(include="all")

,index,text,labels,spacy_doc,len_words,len_sents
count,100000.000000,100000,100000.000000,100000,100000.000000,100000.000000
unique,NaN,99963,NaN,100000,NaN,NaN
top,NaN,DRAFT SCOUT -Ratings Index\n\n-Player News Wir...,NaN,"(HARVARD, 'S, Dani, Rodrik, ,, a, favourite, o...",NaN,NaN
freq,NaN,21,NaN,1,NaN,NaN
mean,149999.500000,NaN,0.500000,NaN,560.041140,22.242920
std,125831.203078,NaN,0.500003,NaN,324.066777,14.769812
min,0.000000,NaN,0.000000,NaN,2.000000,1.000000
25%,24999.750000,NaN,0.000000,NaN,262.000000,10.000000
50%,149999.500000,NaN,0.500000,NaN,543.000000,20.000000
75%,274999.250000,NaN,1.000000,NaN,911.000000,33.000000


In [0]:
subset_100k[subset_100k["labels"]==0].describe()

,index,labels,len_words,len_sents
count,50000.000000,50000.0,50000.000000,50000.000000
mean,24999.500000,0.0,522.453340,21.101920
std,14433.901067,0.0,324.362722,14.931696
min,0.000000,0.0,10.000000,1.000000
25%,12499.750000,0.0,216.000000,8.000000
50%,24999.500000,0.0,490.000000,18.000000
75%,37499.250000,0.0,878.000000,32.000000
max,49999.000000,0.0,2211.000000,162.000000


In [0]:
subset_100k[subset_100k["labels"]==1].describe()

,index,labels,len_words,len_sents
count,50000.000000,50000.0,50000.000000,50000.000000
mean,274999.500000,1.0,597.628940,23.383920
std,14433.901067,0.0,319.380235,14.516876
min,250000.000000,1.0,2.000000,1.000000
25%,262499.750000,1.0,313.000000,12.000000
50%,274999.500000,1.0,592.000000,21.000000
75%,287499.250000,1.0,943.000000,34.000000
max,299999.000000,1.0,4301.000000,257.000000


In [0]:
dir(subset_100k["spacy_doc"][0])

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_bulk_merge',
 '_py_tokens',
 '_realloc',
 '_vector',
 '_vector_norm',
 'cats',
 'char_span',
 'count_by',
 'doc',
 'ents',
 'extend_tensor',
 'from_array',
 'from_bytes',
 'from_disk',
 'get_extension',
 'get_lca_matrix',
 'has_extension',
 'has_vector',
 'is_nered',
 'is_parsed',
 'is_sentenced',
 'is_tagged',
 'lang',
 'lang_',
 'mem',
 'merge',
 'noun_chunks',
 'noun_chunks_iterator',
 'print_tree',
 'remove_extension',
 'retokenize',
 'sentiment',
 'sents',
 'set_extension',
 'similarity',
 'tensor',
 'text',
 'text_with_ws',
 'to_array',
 'to_byte

14303